![Photo by Stephen Phillips - Hostreviews.co.uk on UnSplash](https://cf.bstatic.com/xdata/images/hotel/max1024x768/408003083.jpg?k=c49b5c4a2346b3ab002b9d1b22dbfb596cee523b53abef2550d0c92d0faf2d8b&o=&hp=1){fig-align="center" width=50%}


# Import data

In [11]:
import time
from pathlib import Path

import numpy as np
import pandas as pd
from data import pre_process, utils
from lets_plot import *
from lets_plot.bistro.corr import *
from lets_plot.mapping import as_discrete

LetsPlot.setup_html()

**Goal**:


# Data Inspection

In [2]:
df = pd.read_parquet(
    utils.Configuration.INTERIM_DATA_PATH.joinpath(
        "2023-09-27_Processed_dataset_for_NB_use.parquet.gzip"
    )
)
df.head().style.set_sticky(axis=0)

,as_built_plan,available_as_of,basement,bathrooms,bedroom_1_surface,bedroom_2_surface,bedroom_3_surface,bedrooms,building_condition,co2_emission,cadastral_income,connection_to_sewer_network,construction_year,covered_parking_spaces,dining_room,double_glazing,energy_class,external_reference,flood_zone_type,furnished,garden_surface,gas_water__electricity,heating_type,kitchen_surface,kitchen_type,latest_land_use_designation,living_area,living_room_surface,number_of_frontages,office,outdoor_parking_spaces,planning_permission_obtained,possible_priority_purchase_right,price,primary_energy_consumption,proceedings_for_breach_of_planning_regulations,reference_number_of_the_epc_report,street_frontage_width,subdivision_permit,surface_of_the_plot,surroundings_type,tv_cable,tenement_building,toilets,website,width_of_the_lot_on_the_street,yearly_theoretical_total_energy_consumption,ad_url,day_of_retrieval,city,house_number,zip,street
0,nan,After signing the deed,1.000000,4.000000,28.000000,24.000000,21.000000,5.000000,Good,74.000000,7615.000000,0.000000,1975.000000,2.000000,1.000000,1.000000,D,5530019,nan,0.000000,4300.000000,1.000000,Fuel oil,21.000000,USA hyper equipped,None,420.000000,63.000000,4.000000,1.000000,2.000000,nan,nan,1225000.000000,296.000000,nan,20210910012683,23.000000,nan,4677.000000,"Living area (residential, urban or rural)",1.000000,0.000000,5.000000,http://www.bytheway.be,nan,133062.000000,https://www.immoweb.be/en/classified/villa/for-sale/braine-l%27alleud/1420/10844811,2023-09-27 14:02:10.370650,Waterloo,96,1410,drve richelle
1,0.000000,After signing the deed,nan,1.000000,14.000000,10.000000,nan,2.000000,To renovate,76.000000,541.000000,1.000000,1850.000000,1.000000,1.000000,1.000000,D,5246367,nan,0.000000,nan,nan,Fuel oil,nan,Semi equipped,"Living area (residential, urban or rural)",113.000000,nan,2.000000,nan,nan,nan,0.000000,89000.000000,307.000000,0.000000,20230306014621,9.000000,0.000000,73.000000,Urban,1.000000,0.000000,1.000000,http://www.weinvest.be,10.000000,43587.000000,https://www.immoweb.be/en/classified/town-house/for-sale/jemelle/5580/10843929,2023-09-27 14:02:10.641758,Famenne,65,6900,Chausse de lOurthe
2,nan,After signing the deed,1.000000,1.000000,17.000000,11.000000,nan,2.000000,To renovate,173.000000,689.000000,1.000000,1949.000000,1.000000,1.000000,1.000000,G,5534704,0.000000,0.000000,315.000000,nan,Fuel oil,13.000000,Semi equipped,"Living area (residential, urban or rural)",139.000000,10.000000,3.000000,1.000000,nan,nan,0.000000,150000.000000,699.000000,0.000000,20230303013078,18.000000,0.000000,413.000000,Isolated,1.000000,0.000000,1.000000,http://www.nigel-immo.be,18.000000,96913.000000,https://www.immoweb.be/en/classified/house/for-sale/esneux%20tilff/4130/10838582,2023-09-27 14:02:10.905808,Oupeye,2A,4680,Rue de la Wallonie
3,nan,After signing the deed,nan,2.000000,14.000000,14.000000,10.000000,3.000000,Good,4167.000000,898.000000,1.000000,1899.000000,nan,nan,1.000000,C,5535455,0.000000,0.000000,nan,1.000000,Gas,17.000000,USA installed,"Living area (residential, urban or rural)",119.000000,14.000000,2.000000,nan,nan,1.000000,1.000000,272000.000000,246.000000,0.000000,20230622-0002923649-RES-1,4.000000,0.000000,55.000000,Urban,1.000000,0.000000,2.000000,http://www.immodavinci.be,nan,nan,https://www.immoweb.be/en/classified/house/for-sale/gent/9000/10838392,2023-09-27 14:02:11.811691,Gent,1,9000,SintDenijslaan
4,nan,At delivery,nan,1.000000,16.000000,12.000000,12.000000,3.000000,As new,nan,nan,1.000000,2021.000000,nan,nan,1.000000,A,5527171,0.000000,0.000000,250.000000,1.000000,Gas,nan,USA hyper equipped,"Living area (residential, urban or rural)",215.000000,46.000000,3.000000,nan,1.000000,1.000000,0.000000,413150.000000,27.000000,nan,12345,7.000000,1.000000,330.000000,"Living area (residential, urban or rural)",1.000000,0.000000,2.000000,http://www.living-stone.be,nan,nan,https://www.immoweb.be/en/classified/house/for-sale/affligem/1790/10842333,2023-09-27 14:02:11.968969,Dilbeek,1,1700,Astrid

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3660 entries, 0 to 3905
Data columns (total 53 columns):
 #   Column                                          Non-Null Count  Dtype         
---  ------                                          --------------  -----         
 0   as_built_plan                                   1722 non-null   float64       
 1   available_as_of                                 2134 non-null   object        
 2   basement                                        1707 non-null   float64       
 3   bathrooms                                       3472 non-null   float32       
 4   bedroom_1_surface                               2495 non-null   float32       
 5   bedroom_2_surface                               2439 non-null   float32       
 6   bedroom_3_surface                               2005 non-null   float32       
 7   bedrooms                                        3641 non-null   float32       
 8   building_condition                              3515 

# Assessing Feature Cardinality

We'll now examine the feature cardinality of our dataset to distinguish between categorical and numerical data. This analysis will help us categorize and work with these features effectively.

In [4]:
# Assuming df is your DataFrame
number_unique_entries = {
    "column_name": df.columns.tolist(),
    "column_dtype": [df[col].dtype for col in df.columns],
    "unique_values_pct": [df[col].nunique() for col in df.columns],
}

(
    pd.DataFrame(number_unique_entries)
    .sort_values("unique_values_pct")
    .assign(
        unique_values_pct=lambda x: x.unique_values_pct.div(df.shape[0])
        .mul(100)
        .round(1)
    )
    .pipe(
        lambda df: ggplot(df, aes("unique_values_pct", "column_name"))
        + geom_bar(stat="identity", orientation="y")
        + labs(
            title="Assessing Feature Cardinality",
            subtitle=""" Features with a Low Cardinality (Less than 10 Distinct Values) Can Be  Utilized as Categorical Variables, 
            while Those with Higher Cardinality, typically represented as floats or integers, May Be Employed as They Are
            """,
            x="Percentage of Unique Values per Feature",
            y="",
            caption="https://www.immoweb.be/",
        )
        + theme(
            plot_subtitle=element_text(
                size=12, face="italic"
            ),  # Customize subtitle appearance
            plot_title=element_text(size=15, face="bold"),  # Customize title appearance
        )
        + ggsize(800, 1000)
    )
)

In [5]:
(
    pd.DataFrame(number_unique_entries)
    .sort_values("unique_values_pct")
    .pipe(
        lambda df: ggplot(df, aes("unique_values_pct", "column_name"))
        + geom_bar(stat="identity", orientation="y")
        + labs(
            title="Assessing Feature Cardinality",
            subtitle=""" Features with a Low Cardinality (Less than 10 Distinct Values) Can Be  Utilized as Categorical Variables, 
            while Those with Higher Cardinality, typically represented as floats or integers, May Be Employed as They Are
            """,
            x="Number of Unique Values per Feature",
            y="",
            caption="https://www.immoweb.be/",
        )
        + theme(
            plot_subtitle=element_text(
                size=12, face="italic"
            ),  # Customize subtitle appearance
            plot_title=element_text(size=15, face="bold"),  # Customize title appearance
        )
        + ggsize(800, 1000)
    )
)

# Looking at distributions
## Low cardinality features

In [6]:
low_cardinality_features = (
    pd.DataFrame(number_unique_entries)
    .query("unique_values_pct <= 5")
    .column_name.to_list()
)

['as_built_plan',
 'available_as_of',
 'basement',
 'connection_to_sewer_network',
 'dining_room',
 'double_glazing',
 'flood_zone_type',
 'furnished',
 'gas_water__electricity',
 'number_of_frontages',
 'office',
 'planning_permission_obtained',
 'possible_priority_purchase_right',
 'proceedings_for_breach_of_planning_regulations',
 'subdivision_permit',
 'tv_cable',
 'tenement_building']

In [45]:
high_cardinality_features = (
    pd.DataFrame(number_unique_entries)
    .query("(unique_values_pct >= 5)")
    .loc[lambda df: (df.column_dtype == "float32") | (df.column_dtype == "float64"), :]
    .column_name.to_list()
)

['bathrooms',
 'bedroom_1_surface',
 'bedroom_2_surface',
 'bedroom_3_surface',
 'bedrooms',
 'co2_emission',
 'cadastral_income',
 'construction_year',
 'covered_parking_spaces',
 'garden_surface',
 'kitchen_surface',
 'living_area',
 'living_room_surface',
 'number_of_frontages',
 'outdoor_parking_spaces',
 'price',
 'primary_energy_consumption',
 'street_frontage_width',
 'surface_of_the_plot',
 'toilets',
 'width_of_the_lot_on_the_street',
 'yearly_theoretical_total_energy_consumption']

In [7]:
(
    df.melt(id_vars=["ad_url", "price"])
    .dropna()
    .loc[lambda df: df.variable == "as_built_plan", :]
    # .query("variable.isin(@feature)")
)

,ad_url,price,variable,value
1,https://www.immoweb.be/en/classified/town-hous...,89000.0,as_built_plan,0.0
6,https://www.immoweb.be/en/classified/house/for...,229000.0,as_built_plan,0.0
7,https://www.immoweb.be/en/classified/house/for...,149000.0,as_built_plan,0.0
12,https://www.immoweb.be/en/classified/house/for...,274000.0,as_built_plan,0.0
17,https://www.immoweb.be/en/classified/house/for...,1050000.0,as_built_plan,1.0
...,...,...,...,...
3644,https://www.immoweb.be/en/classified/house/for...,230000.0,as_built_plan,0.0
3647,https://www.immoweb.be/en/classified/house/for...,230000.0,as_built_plan,0.0
3651,https://www.immoweb.be/en/classified/house/for...,230000.0,as_built_plan,0.0
3656,https://www.immoweb.be/en/classified/house/for...,215000.0,as_built_plan,0.0


In [8]:
plots = []

for idx, feature in enumerate(low_cardinality_features):
    plot = (
        df.melt(id_vars=["ad_url", "price"])
        .loc[lambda df: df.variable == feature, :]
        .assign(price=lambda df: np.log10(df.price))
        .pipe(
            lambda df: ggplot(
                df,
                aes(
                    as_discrete("value"),
                    "price",
                ),
            )
            + facet_wrap("variable")
            + geom_boxplot(
                show_legend=False,
            )
        )
    )
    plots.append(plot)
gggrid(plots, ncol=4)

# Correlations using heatmap

In [46]:
(
    df
    # .select_dtypes("number")
    .loc[:, lambda df: df.columns.isin(high_cardinality_features)].pipe(
        lambda df: corr_plot(df)
        .points("full")
        .labels(type="full")
        .palette_gradient(low="#2986cc", mid="#ffffff", high="#d73027")
        .build()
    )
)